In [1]:
import pandas as pd
import numpy as np
import folium
from bokeh.io import output_notebook, show, output_file
output_notebook()

Loading BokehJS ...

In [2]:
confirmed_time = pd.read_csv('./data/confirmed_over_time.csv')

In [3]:
confirmed_time['DATE'] = pd.to_datetime(confirmed_time['DATE'])

In [4]:
confirmed_time

,DATE,CONFIRMED
0,2020-01-22,332
1,2020-01-23,600
2,2020-01-24,915
3,2020-01-25,1971
4,2020-01-26,2726
5,2020-01-27,2811
6,2020-01-28,5946
7,2020-01-29,7647
8,2020-01-30,9620
9,2020-01-31,11181


In [5]:
from bokeh.plotting import figure

p = figure(x_axis_type="datetime", title="Confirmed Wuhan Coronavirus Cases", plot_height=350, plot_width=730)

# modify properties
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Confirmed Cases'

# assign mark
p.line(confirmed_time.DATE, confirmed_time.CONFIRMED)

p.toolbar_location = None
p.toolbar.logo = None
show(p)

In [6]:
output_file('./data/cases_over_time.html')
show(p)

In [7]:
cases = pd.read_csv('./data/2_1_counts.csv')

In [8]:
cases.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,lat,lon
0,Hubei,Mainland China,2/1/2020 23:33,9074,294,215,31.151725,112.878322
1,Zhejiang,Mainland China,2/2/2020 2:13,661,0,23,29.000000,120.000000
2,Guangdong,Mainland China,2/2/2020 1:23,604,0,14,23.135769,113.198269
3,Henan,Mainland China,2/2/2020 0:53,493,2,4,34.000000,114.000000
4,Hunan,Mainland China,2/2/2020 1:53,463,0,8,27.666209,111.748706


In [9]:
import math
import branca

max_cases = math.sqrt(cases.Confirmed.max())
min_cases = math.sqrt(cases.Confirmed.min())

In [10]:
my_map = folium.Map(
    location=[20,0],
    zoom_start=2,
    min_zoom=2,
    tiles='cartodbpositron'
)

colorscale = branca.colormap.linear.PuOr_03.scale(min_cases, max_cases)

for index, row in cases.iterrows():
    if (type(cases.iloc[index]['Province/State']) == str):
        row_text = f"{row['Province/State']}, {row['Country/Region']}.<br>Cases confirmed: {row['Confirmed']}.<br> Deaths: {row['Deaths']}<br>Recovered: {row['Recovered']}"
    else:
        row_text = f"{row['Country/Region']}.<br>Cases confirmed: {row['Confirmed']}<br>Deaths: {row['Deaths']}<br>Recovered: {row['Recovered']}"

    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=500000*(math.sqrt(row.Confirmed)/max_cases),
        color=colorscale(row['Deaths']),
        fill=True,
        fill_opacity=1,
        fill_color=colorscale(row['Deaths']),
        popup=folium.Popup(row_text, max_width=300)
        
    ).add_to(my_map)

my_map

In [11]:
my_map.save('./data/corona_virus_map.html')